In [1]:
# On commence par importer les librairies nécessaires pour l'analyse des données.

import duckdb
import pandas as pd

# On importe maintenant les données SISE-eaux, pour l'instant hebergé en "local" dans le GitHub en attendant le travail des data eng

# res: résultats des prélèvements, commune: liste des communes et réseaux UDI, prelev: détails administratifs des prélèvements

res = pd.read_csv(
    r"C:\Users\emma.david\Desktop\data for good\eau test\draft-eau-tests\database\data_temporaire\DIS_RESULT_2024.txt"
)

commune = pd.read_csv(
    r"C:\Users\emma.david\Desktop\data for good\eau test\draft-eau-tests\database\data_temporaire\DIS_COM_UDI_2024.txt"
)

prelev = pd.read_csv(
    r"C:\Users\emma.david\Desktop\data for good\eau test\draft-eau-tests\database\data_temporaire\DIS_PLV_2024.txt"
)

C:\Users\emma.david\AppData\Local\Temp\ipykernel_12588\2228173078.py:10: DtypeWarning: Columns (0,1,16) have mixed types. Specify dtype option on import or set low_memory=False.
  res = pd.read_csv(r"C:\Users\emma.david\Desktop\data for good\eau test\draft-eau-tests\database\data_temporaire\DIS_RESULT_2024.txt")
C:\Users\emma.david\AppData\Local\Temp\ipykernel_12588\2228173078.py:14: DtypeWarning: Columns (0,1,2,4,7) have mixed types. Specify dtype option on import or set low_memory=False.
  prelev = pd.read_csv(r"C:\Users\emma.david\Desktop\data for good\eau test\draft-eau-tests\database\data_temporaire\DIS_PLV_2024.txt")


In [2]:
# On affiche le haut du dataframe res

res.head()

,cddept,referenceprel,cdparametresiseeaux,cdparametre,libmajparametre,libminparametre,libwebparametre,qualitparam,insituana,rqana,cdunitereferencesiseeaux,cdunitereference,limitequal,refqual,valtraduite,casparam,referenceanl
0,1,100138503,ALTMICR,1370.0,ALUMINIUM TOTAL µG/L,Aluminium total µg/l,NaN,N,L,<10,µg/L,133,NaN,<=200 µg/L,0.00,7429-90-5,100147233.0
1,1,100138503,ASP,6489.0,ASPECT (QUALITATIF),Aspect (qualitatif),NaN,O,L,Aspect normal,SANS OBJET,X,NaN,NaN,0.00,NaN,100147232.0
2,1,100138503,ASP,6489.0,ASPECT (QUALITATIF),Aspect (qualitatif),NaN,O,L,Aspect normal,SANS OBJET,X,NaN,NaN,0.00,NaN,100147233.0
3,1,100138503,CL2LIB,1398.0,CHLORE LIBRE,Chlore libre,NaN,N,T,"0,30",mg(Cl2)/L,165,NaN,NaN,0.30,NaN,NaN
4,1,100138503,CL2TOT,1399.0,CHLORE TOTAL,Chlore total,NaN,N,T,"0,34",mg(Cl2)/L,165,NaN,NaN,0.34,NaN,NaN


In [3]:
res.columns

Index(['cddept', 'referenceprel', 'cdparametresiseeaux', 'cdparametre',
       'libmajparametre', 'libminparametre', 'libwebparametre', 'qualitparam',
       'insituana', 'rqana', 'cdunitereferencesiseeaux', 'cdunitereference',
       'limitequal', 'refqual', 'valtraduite', 'casparam', 'referenceanl'],
      dtype='object')

Les colonnes qui vont donc nous intéresser, au moins dans un premier temps, sont: referenceprel, 